# 1. Setup

In [ ]:
import tensorflow as tf
import cv2
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow_docs.modeling import EpochDots
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint 
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled.")
    except RuntimeError as e:
        print(e)

tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3800)]
)

# 2. Load Data

In [ ]:
data_dir = '../data/data_pool'

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir, batch_size=32, image_size=(256, 256), color_mode='rgb', shuffle=True)

# Get number of classes
num_classes = len(data.class_names)
print(f"Number of classes: {num_classes}")

# Get class names
print(f"Class names: {data.class_names}")

# Get total number of files
total_files = len(data.file_paths)
print(f"Total number of files: {total_files}")

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

# 3. Scale Data

In [ ]:
data = data.map(lambda x, y: (x / 255.0, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()
batch[0].max()

# 4. Split Data

In [ ]:
train_size = int(len(data)*0.75)
val_size = int(len(data)*0.17)
test_size = int(len(data)*0.08)

In [ ]:
train_data = data.take(train_size)
val_data = data.skip(train_size).take(val_size)
test_data = data.skip(train_size + val_size).take(test_size)

In [ ]:
print(f"train_size: {train_size}")
print(f"val_size: {val_size}")
print(f"test_size: {test_size}")

print(f"train_data: {len(train_data)}")
print(f"val_data: {len(val_data)}")
print(f"test_data: {len(test_data)}")

# 5. Build Convolutional Neural Network

In [ ]:
# Input shape constants
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3

# Much smaller L2 regularization
reg_factor = 0.001

In [ ]:
model = Sequential([
    # Input Layer
    Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),    

    # First Convolutional Block - halved filters (16 instead of 32)
    Conv2D(16, (3, 3), padding='same', activation='relu',
           kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Conv2D(16, (3, 3), padding='same', activation='relu',
           kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Second Convolutional Block - halved filters (32 instead of 64)
    Conv2D(32, (3, 3), padding='same', activation='relu',
           kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Conv2D(32, (3, 3), padding='same', activation='relu',
           kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    # Third Convolutional Block - halved filters (64 instead of 128)
    Conv2D(64, (3, 3), padding='same', activation='relu',
           kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    Conv2D(64, (3, 3), padding='same', activation='relu',
           kernel_regularizer=l2(reg_factor)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    # Flatten the output and add dense layers
    Flatten(),

    # Reduced number of dense layers with multiples of 20 nodes
    Dense(256, activation='relu', kernel_regularizer=l2(reg_factor)),  # 16x number of classes
    BatchNormalization(),
    Dropout(0.3),  # Reduced dropout 

    # Reduced number of dense layers with multiples of 20 nodes
    Dense(128, activation='relu', kernel_regularizer=l2(reg_factor)),  # 16x number of classes
    BatchNormalization(),
    Dropout(0.3),  # Reduced dropout 

    # Reduced number of dense layers with multiples of 20 nodes
    Dense(128, activation='relu', kernel_regularizer=l2(reg_factor)),  # 16x number of classes
    BatchNormalization(),
    Dropout(0.3),  # Reduced dropout 

    Dense(64, activation='relu', kernel_regularizer=l2(reg_factor)),  # 8x number of classes
    BatchNormalization(),
    Dropout(0.3),  # Reduced dropout
    
    Dense(64, activation='relu', kernel_regularizer=l2(reg_factor)),  # 2x number of classes
    BatchNormalization(),
    Dropout(0.3),
    
    # Output layer
    Dense(54, activation='softmax')
])

In [ ]:
optimizer = Adam(learning_rate=0.001)

In [ ]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
model.summary()

# 7. Train model

In [ ]:
# Enhanced training configuration for longer training
training_config = {
    'epochs': 100,  # Increased epochs
    'callbacks': [
        EarlyStopping(
            monitor='val_loss',
            patience=7,  # Increased patience
            restore_best_weights=True,
            min_delta=0.001  # Smaller improvement threshold
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,  # Increased patience
            min_lr=1e-6,  # Lower minimum learning rate
            min_delta=0.001
        ),
        ModelCheckpoint(
            filepath='../data/models/checkpoint.model.keras',
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        TensorBoard(r'../data/logs')
    ]
}

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=training_config['epochs'],
    callbacks=training_config['callbacks']
)

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Save Model

In [ ]:
model_dir = os.path.join('..', 'data', 'models')
os.makedirs(model_dir, exist_ok=True)

In [ ]:
model.save(os.path.join(model_dir, 'uno_classifier.h5'))

# 10. Test model

In [ ]:
model_path = os.path.join(model_dir, 'uno_classifier.h5')
model = load_model(model_path)

In [ ]:
img = cv2.imread('test_image.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.title("Original Image")
plt.axis('off')
plt.show()

In [ ]:
# Resize and scale the image
resize = tf.image.resize(img_rgb, (256, 256))
scaled_img = resize.numpy().astype(np.float32) / 255.0
plt.imshow(scaled_img)
plt.title("Preprocessed Image")
plt.axis('off')
plt.show()

In [ ]:
# Expand dimensions to match the model's input shape
input_img = np.expand_dims(scaled_img, axis=0)

In [ ]:
# Make prediction
yhat = model.predict(input_img)
predicted_index = np.argmax(yhat, axis=1)[0]
predicted_label = data.class_names[predicted_index]

print(f"Predicted Class Index: {predicted_index}")
print(f"Predicted Class Name: {predicted_label}")

In [ ]:
yhat

In [ ]:
# Verify if the prediction matches the expected class
expected_index = 31  # 31st class (0-based index)
expected_label = data.class_names[expected_index]

if predicted_index == expected_index:
    print(f"The model correctly predicted the class: {predicted_label}")
else:
    print(f"The model predicted '{predicted_label}', but expected '{expected_label}'.")